<a href="https://colab.research.google.com/github/ladyada/notebooks/blob/master/TFLiteConvert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1.) Install Bazel**


In [0]:
BAZEL_VERSION = '0.20.0'
!wget https://github.com/bazelbuild/bazel/releases/download/{BAZEL_VERSION}/bazel-{BAZEL_VERSION}-installer-linux-x86_64.sh
!chmod +x bazel-{BAZEL_VERSION}-installer-linux-x86_64.sh
!./bazel-{BAZEL_VERSION}-installer-linux-x86_64.sh
!bazel

**2) Install Object Detection Model & SSDLite**

In [0]:

!git clone --depth=1 --recurse-submodules https://github.com/tensorflow/models.git
!cd /content/models/research && protoc object_detection/protos/*.proto --python_out=.
%cd /content/models/research/object_detection
checkpoint_name = 'ssdlite_mobilenet_v2_coco_2018_05_09'
!wget http://download.tensorflow.org/models/object_detection/{checkpoint_name}.tar.gz
!tar -xzf {checkpoint_name}.tar.gz
checkpoint = '{0}.ckpt'.format(checkpoint_name)
print('Setup successful!')

**3) Export TFLite graph**

In [0]:
import os
os.environ['PYTHONPATH'] += ":/content/models/research/:/content/models/research/slim"
!pwd
!ls -la
!python export_tflite_ssd_graph.py \
 --pipeline_config_path={checkpoint_name}/pipeline.config \
 --trained_checkpoint_prefix={checkpoint_name}/model.ckpt \
 --output_directory=/content/tmp \
 --add_postprocessing_op=true


**3.) Convert .pb model into TFLite .lite model**

In [0]:
%cd /content/models/
!ls
!bazel run -c opt //tensorflow/lite/toco:toco -- \
--input_file=/content/tmp/tflite_graph.pb \
--output_file=/content/tmp/detect.tflite \
--input_shapes=1,300,300,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
--inference_type=FLOAT \
--allow_custom_ops

**4.) download optimized .lite file to local machine**

In [0]:
files.download(tflite_file)